<h1>Web Scraping</h1>

cuando no nos dan la opción de descargar los datos directamente en un archivo, tenemos la opción de descargarlos directamente mediante la técnica de web scraping.
Haremos el ejemplo con la web de aemet. Aunque esta web permita exportar los archivos vamos a obviarlo y hacer la exportación solo de la tabla.
La solución de copiar y pegar típica, control c y control v no es buena opción.
Web scraping se refiere a la descarga de datos de una web de forma legible.

Se puede hacer mediante la utilización de API o con cóidgo de python o otras acciones.
Cuando las web no tienen apis o estas proporcioan datos que no nos interesan  una buena opción es crear un rastreador web mediante python

Ojo, hay webs que tienen licencias que impiden obtener datos mediante técnicas de web scrappig por lo que eso debe tenerse en cuenta

Se puede inspeccionar la web y en la pestaña elements, accedes al código html. Vamos navegando por el inspector hasta llegar a la tag table

In [1]:
# Antes de importar las siguientes librerías, debemos instalarlas si no las tenemos con pill install requests y pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://www.aemet.es/es/eltiempo/observacion/ultimosdatos?k=ast&l=1249X&w=0&datos=det&x=&f=temperatura"

In [3]:
html_content = requests.get(url).text
# request permite acceder una url. mediante el método get (url) se extrae la información de ella y la propiedad text, para que solo devuelva texto
html_content

#Se observa como se han traido los datos de la web, pero sin ningún tipo de formato. hay que seguir trabajando en ellos para darles el formato deseado.

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml" lang="es" xml:lang="es"><head>\r\n<meta name="description" content="El Tiempo. Datos horarios y resúmenes diarios de la red de estaciones automáticas: Oviedo - Tabla" />\r\n<meta name="keywords" content="AEMET,Estatal,Meteorología,España,Agencia,Nacional,El Tiempo,Tiempo,observación,datos,últimos,resumen,tiempo,estaciones,automáticas,temperatura,humedad,viento,precipitación,presión,Oviedo,Asturias" />\r\n<meta name="author" content="Agencia Estatal de Meteorología" />\r\n\r\n<meta content="width=device-width, initial-scale=1.0" name="viewport" />\r\n<meta name="title" content="El Tiempo. Hoy y últimos días: Oviedo - Datos horarios - Tabla - Agencia Estatal de Meteorología - AEMET. Gobierno de España" />\r\n<meta name="DC.title" content="El Tiempo. Hoy y últimos días: Oviedo - Datos horarios - Tabla - Agencia Estatal de 

In [4]:
# vamos a hacer una extracción un poco más organizada con beautifullSoup, que nos lo devolverá con las etiquetas y más organizado.
soup = BeautifulSoup(html_content,"lxml")
soup

"""
Puede que de problemas, es posible que necesitemos instalar el parseador lxml.
el parámetro "lxml" del constructor de BeautifulSoup puede dar problemas, es posible que sin ponerlo funcione
En BeautifulSoup, el parámetro "lxml" se refiere al analizador (parser) que se utilizará para analizar el HTML o XML.

"Lxml" es uno de los analizadores disponibles en BeautifulSoup y utiliza la biblioteca lxml para el análisis.
Lxml es conocido por ser rápido y eficiente, y es una opción popular para analizar documentos HTML/XML en BeautifulSou

"""

'\nPuede que de problemas, es posible que necesitemos instalar el parseador lxml.\nel parámetro "lxml" del constructor de BeautifulSoup puede dar problemas, es posible que sin ponerlo funcione\nEn BeautifulSoup, el parámetro "lxml" se refiere al analizador (parser) que se utilizará para analizar el HTML o XML.\n\n"Lxml" es uno de los analizadores disponibles en BeautifulSoup y utiliza la biblioteca lxml para el análisis.\nLxml es conocido por ser rápido y eficiente, y es una opción popular para analizar documentos HTML/XML en BeautifulSou\n\n'

In [5]:
#metodo prettyfy() anida cada tag para verse mejor
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="es" xml:lang="es" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="El Tiempo. Datos horarios y resúmenes diarios de la red de estaciones automáticas: Oviedo - Tabla" name="description"/>
  <meta content="AEMET,Estatal,Meteorología,España,Agencia,Nacional,El Tiempo,Tiempo,observación,datos,últimos,resumen,tiempo,estaciones,automáticas,temperatura,humedad,viento,precipitación,presión,Oviedo,Asturias" name="keywords"/>
  <meta content="Agencia Estatal de Meteorología" name="author"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="El Tiempo. Hoy y últimos días: Oviedo - Datos horarios - Tabla - Agencia Estatal de Meteorología - AEMET. Gobierno de España" name="title"/>
  <meta content="El Tiempo. Hoy y últimos días: Oviedo - Datos horarios - Tabla - Agencia Estatal de Meteorología - AEMET. Gobierno de

In [6]:
# ahora vamos a extraer secciones o etiquetas concretas, por ejemplo el elemento html title
soup.title

<title>El Tiempo. Hoy y últimos días: Oviedo - Datos horarios - Tabla - Agencia Estatal de Meteorología - AEMET. Gobierno de España</title>

In [7]:
soup.title.text
#solo queremos extraer el texto de la etiqueta title

'El Tiempo. Hoy y últimos días: Oviedo - Datos horarios - Tabla - Agencia Estatal de Meteorología - AEMET. Gobierno de España'

In [8]:
#Ahora vamos a extraer datos de la tabla en la variable weather_table. Para ello vamos a buscar el atributo class del elemento <table> 
weather_table = soup.find("table", attrs={"class":"width100 tabla_dinamica tabla_responsive"}) #lo que decimos aquí es que queremos que encuentre una tabla, y con el parámetro attrs precisamos pasándole un diccionario, con la clave el nombre del atributo y el valor, el valor del atributo del elemento html. que tenga como atributo una class, con el valor del atributo clase "width100 tabla_dinamica tabla_responsive"
weather_table

<table cellspacing="0" class="width100 tabla_dinamica tabla_responsive" id="table" summary="Últimos datos meteorológicos observados por cada ciudad de la comunidad autónoma seleccionada">
<thead>
<tr class="height50px">
<th abbr="Fecha y hora oficial">
<div class="cont_h3_th_sup">
<div>Fecha y hora oficial</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Temp." title="Temperatura (°C)">
<div class="cont_h3_th_sup">
<div>Temp. (°C)</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="V. vien." title="Velocidad del viento (km/h)">
<div class="cont_h3_th_sup">
<div>V. vien. (km/h)</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Dir. viento" title="Dirección del viento">
<div class="cont_h3_th_sup">
<div>Dir. viento</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Racha" title="Racha (km/h)">
<div class="cont_h3_th_sup">
<div>Racha (km/h)</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div

In [9]:
#Necesitaremos extraer el nombre de las columnas, es decir el encabezado de la tabla, porque serán 
table_header = weather_table.thead
table_header

<thead>
<tr class="height50px">
<th abbr="Fecha y hora oficial">
<div class="cont_h3_th_sup">
<div>Fecha y hora oficial</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Temp." title="Temperatura (°C)">
<div class="cont_h3_th_sup">
<div>Temp. (°C)</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="V. vien." title="Velocidad del viento (km/h)">
<div class="cont_h3_th_sup">
<div>V. vien. (km/h)</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Dir. viento" title="Dirección del viento">
<div class="cont_h3_th_sup">
<div>Dir. viento</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Racha" title="Racha (km/h)">
<div class="cont_h3_th_sup">
<div>Racha (km/h)</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Dir. racha" title="Dirección de racha">
<div class="cont_h3_th_sup">
<div>Dir. racha</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Prec." titl

In [10]:
# Ahora solo queremos la tag asociada a la fila, es decir, necesitamos buscar lo que tiene el tr
header_row = table_header.tr
header_row

<tr class="height50px">
<th abbr="Fecha y hora oficial">
<div class="cont_h3_th_sup">
<div>Fecha y hora oficial</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Temp." title="Temperatura (°C)">
<div class="cont_h3_th_sup">
<div>Temp. (°C)</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="V. vien." title="Velocidad del viento (km/h)">
<div class="cont_h3_th_sup">
<div>V. vien. (km/h)</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Dir. viento" title="Dirección del viento">
<div class="cont_h3_th_sup">
<div>Dir. viento</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Racha" title="Racha (km/h)">
<div class="cont_h3_th_sup">
<div>Racha (km/h)</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Dir. racha" title="Dirección de racha">
<div class="cont_h3_th_sup">
<div>Dir. racha</div>
<div class="cont_h3_th_inf"><h3> </h3></div>
</div>
</th>
<th abbr="Prec." title="Preci

In [11]:
"""
Ahora nos interesan las etiquetas th, y estas tienen un atributo que se llama title, que es el nombre de la etiqueta.
para no hacerlos uno a uno, necesitarmeos hacer un bucle para ir recorriendo cada una de ellas y recuperando el valor de lo que nos interesa
esto lo guardaremos en una variable llamada col_names = list () que la iniciaremos como una lista vacia y se irá rellenando con los nombres de las columnas


"""

col_names = list()

for th in header_row.find_all("th"): #Porque ahora queremos todas las th y no solo una
    col_names.append(th.text)  #append es el método para añadir elementos a una lista

col_names


['\n\nFecha y hora oficial\n\xa0\n\n',
 '\n\nTemp. (°C)\n\xa0\n\n',
 '\n\nV. vien. (km/h)\n\xa0\n\n',
 '\n\nDir. viento\n\xa0\n\n',
 '\n\nRacha (km/h)\n\xa0\n\n',
 '\n\nDir. racha\n\xa0\n\n',
 '\n\nPrec. (mm)\n\xa0\n\n',
 '\n\nPresión (hPa)\n\xa0\n\n',
 '\n\nTend. (hPa)\n\xa0\n\n',
 '\n\nHumedad (%)\n\xa0\n\n']

In [12]:
# Hemos obtenido los nombres de las columnas, pero en un formato que no es el adecuado para nosotros. por lo que utilizaremos el método replace, que se usa mucho para que cuando encuentre \n lo reemplace por nada
col_names = list()

for th in header_row.find_all("th"): #Porque ahora queremos todas las th y no solo una
    col_names.append(th.text.replace("\n",""))  #append es el método para añadir elementos a una lista

col_names

['Fecha y hora oficial\xa0',
 'Temp. (°C)\xa0',
 'V. vien. (km/h)\xa0',
 'Dir. viento\xa0',
 'Racha (km/h)\xa0',
 'Dir. racha\xa0',
 'Prec. (mm)\xa0',
 'Presión (hPa)\xa0',
 'Tend. (hPa)\xa0',
 'Humedad (%)\xa0']

In [13]:
col_names = list()

for th in header_row.find_all("th"): #Porque ahora queremos todas las th y no solo una
    col_names.append(th.text.replace("\n","").strip("\xa0"))  #con strip trimamos los caracteres que indicamos

col_names

"""
Con esto ya tendríamos los nombres de las columnas en el formato adecuado
"""

'\nCon esto ya tendríamos los nombres de las columnas en el formato adecuado\n'

In [14]:
"""
Ahora debemos obtener los datos del cuerpo, ya que ya tenemos los del encabezado

"""

table_body = weather_table.tbody
table_body

<tbody>
<tr>
<td>11/06/2023 01:00</td>
<td class="fila_impar">16.7</td>
<td class="fila_impar">0</td>
<td class="fila_impar"><span class="nodisplay">1-Calma</span><img alt="Calma" src="/imagenes/png/iconos_viento_udat/C.png" title="Calma"/></td>
<td class="fila_impar">7</td>
<td class="fila_impar"><span class="nodisplay">6-Sur</span><img alt="Sur" src="/imagenes/png/iconos_viento_udat/S.png" title="Sur"/></td>
<td class="fila_impar">0.0</td>
<td class="fila_impar">977.8</td>
<td class="fila_impar">0.6</td>
<td class="fila_impar">94.0</td>
</tr>
<tr>
<td>11/06/2023 00:00</td>
<td class="fila_impar">16.7</td>
<td class="fila_impar">5</td>
<td class="fila_impar"><span class="nodisplay">5-Sudeste</span><img alt="Sudeste" src="/imagenes/png/iconos_viento_udat/SE.png" title="Sudeste"/></td>
<td class="fila_impar">10</td>
<td class="fila_impar"><span class="nodisplay">4-Este</span><img alt="Este" src="/imagenes/png/iconos_viento_udat/E.png" title="Este"/></td>
<td class="fila_impar">0.0</td>


ahora necesitamos crear un bucle desde este momento porque ahora tenemos que extraer más de una fila, no como en el encabezado


In [15]:
len(table_body.find_all("tr")) # Permite calcular el número de filas que tiene la tabla. fin_all es para encontrar todas

24

In [16]:
# cremos variable para ir creando las filas en forma d elista -- table_rows = [] que se ira rellenando por cada iteración del bucle
table_rows = [] # también se puede incializar así table_rows = list()
for trow in table_body.find_all("tr"):
    row=[] # creamos una lista para esa fila, porque cada fila tiene varias cosas, varias td. En cada iteración estamos dentro de una fila completa, con varios td
    for td in trow.find_all("td"): # bucle anidado para que en cada elemento tr, recorra internamente todas las td que contiene, y que las vaya añadiendo a la variable lista row
        row.append(td.text)
        
    table_rows.append(row) #para cada vuelta de tr vas a añadir lo que contiene la variable row, que son todas las td de cada fila que se ha recorrido en el bucle anidado y que se encuentra en el primer nivel del bucle, no en el anidado

table_rows  # aquí ya se pueden ver todos los datos td , aunque en vertical. Es decir, tenemos una lista para cada una de las filas del cuerpo de la tabla

[['11/06/2023 01:00',
  '16.7',
  '0',
  '1-Calma',
  '7',
  '6-Sur',
  '0.0',
  '977.8',
  '0.6',
  '94.0'],
 ['11/06/2023 00:00',
  '16.7',
  '5',
  '5-Sudeste',
  '10',
  '4-Este',
  '0.0',
  '977.9',
  '1.0',
  '93.0'],
 ['10/06/2023 23:00',
  '17.0',
  '5',
  '5-Sudeste',
  '11',
  '5-Sudeste',
  '0.4',
  '977.9',
  '1.4',
  '92.0'],
 ['10/06/2023 22:00',
  '17.4',
  '4',
  '6-Sur',
  '10',
  '6-Sur',
  '0.2',
  '977.2',
  '1.0',
  '92.0'],
 ['10/06/2023 21:00',
  '18.1',
  '6',
  '6-Sur',
  '9',
  '6-Sur',
  '0.8',
  '976.9',
  '0.9',
  '90.0'],
 ['10/06/2023 20:00',
  '19.1',
  '6',
  '6-Sur',
  '9',
  '6-Sur',
  '0.4',
  '976.5',
  '0.7',
  '86.0'],
 ['10/06/2023 19:00',
  '19.5',
  '5',
  '6-Sur',
  '17',
  '5-Sudeste',
  '0.0',
  '976.2',
  '0.8',
  '80.0'],
 ['10/06/2023 18:00',
  '19.5',
  '9',
  '3-Nordeste',
  '23',
  '3-Nordeste',
  '0.0',
  '976.0',
  '0.6',
  '72.0'],
 ['10/06/2023 17:00',
  '20.0',
  '10',
  '3-Nordeste',
  '30',
  '3-Nordeste',
  '0.0',
  '975.8',
  

In [17]:
"""la idea de los bucles es que cuando accedamos a una estructura, es decir a un elemento html que tenga anidados otros elementos que necesitemos recorrer utilicemos estos bucles.

El bucle anterior.
Tenemos varias filas dentro de table_body por lo que usamos el método find_all("tr") para encontrar todas las etiquetas tr que estén dentro de table_body. 
Por tanto la primera trow hace referencia a la primera etiqueta tr que hay dentro de la estructura del tbody, en la que la primera tr incluye 

<td>11/06/2023 00:00</td>
<td class="fila_impar">16.7</td>
<td class="fila_impar">5</td>
<td class="fila_impar"><span class="nodisplay">5-Sudeste</span><img alt="Sudeste" src="/imagenes/png/iconos_viento_udat/SE.png" title="Sudeste"/></td>
<td class="fila_impar">10</td>
<td class="fila_impar"><span class="nodisplay">4-Este</span><img alt="Este" src="/imagenes/png/iconos_viento_udat/E.png" title="Este"/></td>
<td class="fila_impar">0.0</td>
<td class="fila_impar">977.9</td>
<td class="fila_impar">1.0</td>
<td class="fila_impar">93.0</td>

una vez que estamos en esta trow, que es lo que contiene la primera tr, hacemos otro bucle para que por cada una de las td haga algo. Cada vez que pase una td la ira añadiendo a la variable row que es de tipo lista y que se ha creado en el bucle padre. En cada vuelta completa del bucle padre, esta variable lista se sobreescribirá con los nuevos td de la siguente fila, pero para no perder esos datos, en el primer bucle, se irán añadiendo a la variable lista que creamos previamente a comenzar los bucles table_rows, que es la lista superior.
"""

'la idea de los bucles es que cuando accedamos a una estructura, es decir a un elemento html que tenga anidados otros elementos que necesitemos recorrer utilicemos estos bucles.\n\nEl bucle anterior.\nTenemos varias filas dentro de table_body por lo que usamos el método find_all("tr") para encontrar todas las etiquetas tr que estén dentro de table_body. \nPor tanto la primera trow hace referencia a la primera etiqueta tr que hay dentro de la estructura del tbody, en la que la primera tr incluye \n\n<td>11/06/2023 00:00</td>\n<td class="fila_impar">16.7</td>\n<td class="fila_impar">5</td>\n<td class="fila_impar"><span class="nodisplay">5-Sudeste</span><img alt="Sudeste" src="/imagenes/png/iconos_viento_udat/SE.png" title="Sudeste"/></td>\n<td class="fila_impar">10</td>\n<td class="fila_impar"><span class="nodisplay">4-Este</span><img alt="Este" src="/imagenes/png/iconos_viento_udat/E.png" title="Este"/></td>\n<td class="fila_impar">0.0</td>\n<td class="fila_impar">977.9</td>\n<td class=

ahora debemos crearnos un dataFrame que lo contenga todo, aunque voy a poner el mport en la primera celda al principo el import pandas as pd

In [18]:
 # Constructor del DataFrame, que nos permitirá imitar la estructura de una tabla, el primer parámetro sera table_row, que es una lista de listas, y el segundo palrametro será el nombre de las columnas, y le proporcionaremos la lista de nombres de las colmnas col_names
    
table = pd.DataFrame(table_rows, columns=col_names)
table

,Fecha y hora oficial,Temp. (°C),V. vien. (km/h),Dir. viento,Racha (km/h),Dir. racha,Prec. (mm),Presión (hPa),Tend. (hPa),Humedad (%)
0,11/06/2023 01:00,16.7,0,1-Calma,7,6-Sur,0.0,977.8,0.6,94.0
1,11/06/2023 00:00,16.7,5,5-Sudeste,10,4-Este,0.0,977.9,1.0,93.0
2,10/06/2023 23:00,17.0,5,5-Sudeste,11,5-Sudeste,0.4,977.9,1.4,92.0
3,10/06/2023 22:00,17.4,4,6-Sur,10,6-Sur,0.2,977.2,1.0,92.0
4,10/06/2023 21:00,18.1,6,6-Sur,9,6-Sur,0.8,976.9,0.9,90.0
5,10/06/2023 20:00,19.1,6,6-Sur,9,6-Sur,0.4,976.5,0.7,86.0
6,10/06/2023 19:00,19.5,5,6-Sur,17,5-Sudeste,0.0,976.2,0.8,80.0
7,10/06/2023 18:00,19.5,9,3-Nordeste,23,3-Nordeste,0.0,976.0,0.6,72.0
8,10/06/2023 17:00,20.0,10,3-Nordeste,30,3-Nordeste,0.0,975.8,0.5,71.0
9,10/06/2023 16:00,20.5,17,3-Nordeste,29,3-Nordeste,0.0,975.4,0.2,68.0


como hemos guardado la web y esto está cogiendo datos directamente desde la web, cada vez que actualicemos el notebook, este obtendrá nuevamente los datos actualizados, que es más cómodo que tener que descargar continuamente el fichero manualmente.
